In [1]:
from pathlib import Path
import sys
sys.path.append('../bitome-kb/')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.utils.class_weight import compute_class_weight
from bitome.core import Bitome
from math import log,exp,inf
bitome = Bitome.init_from_file('../bitome-kb/bitome.pkl')

In [2]:
from feature_functions import *
from pymodulon.io import load_json_model
from pymodulon.visualization import *

The name visualization is deprecated. Please use plotting


In [3]:
shape_table = pd.read_pickle('../../K12DNA_structure/ecoli_U00096/ecoli_U00096_shape.pickle')

In [13]:
import timeit
start = timeit.default_timer()
temp = bitome.get_shape((30,40))
stop = timeit.default_timer()
print('Time: ', stop - start) 
temp

Time:  0.00030229997355490923


,HelT,MGW,ORChLD,ProT,Roll
base,,,,,
30,33.715,5.32,-0.265625,-8.33,0.795
31,33.785,5.14,0.533500,-8.52,0.190
32,34.875,5.02,0.194750,-7.10,1.035
33,35.315,4.99,0.831500,-8.49,-2.335
34,34.030,5.04,-0.070375,-3.46,-1.760
35,33.715,5.00,0.402875,-6.62,-1.465
36,33.545,5.14,0.212500,-3.06,-1.570
37,33.245,5.33,0.528750,-7.00,1.245
38,34.455,5.32,0.445750,-8.12,0.980


In [12]:
# input left and right are all cardinal position
start = timeit.default_timer()
temp = get_seq_shape(bitome.sequence, 30, 40)
stop = timeit.default_timer()
print('Time: ', stop - start) 
temp

Time:  0.08483529998920858


,HelT,MGW,Roll,ProT
cardinal_pos,,,,
30,33.7175,5.32,0.7975,-8.33
31,33.7875,5.14,0.1900,-8.52
32,34.8750,5.02,1.0300,-7.10
33,35.3150,4.99,-2.3400,-8.49
34,34.0300,5.04,-1.7625,-3.46
35,33.7150,5.00,-1.4675,-6.62
36,33.5450,5.14,-1.5725,-3.06
37,33.2450,5.33,1.2425,-7.00
38,34.4550,5.32,0.9775,-8.12


## Load 5mer table

In [6]:
shape5mer_table = {}
for shape_name in ['HelT','MGW','Roll','ProT']:
    shape5mer_table[shape_name] = pd.read_csv('data/shapeTables/'+shape_name+'.5mer.csv',header=None)

In [12]:
shape5mer_table['HelT']

,0,1,2,3,4
0,AAAAA,37.74,0.0,38.01,0.0
1,AAAAC,37.13,0.0,36.95,0.0
2,AAAAG,37.02,0.0,37.18,0.0
3,AAAAT,36.93,0.0,37.68,0.0
4,AAACA,36.59,0.0,35.22,0.0
...,...,...,...,...,...
1019,TTTGT,35.77,0.0,35.41,0.0
1020,TTTTA,36.72,0.0,35.60,0.0
1021,TTTTC,37.00,0.0,36.32,0.0
1022,TTTTG,36.92,0.0,35.51,0.0


In [10]:
temp = [1,2,3,4]
temp[1:-1]

[2, 3]

In [11]:
# check for 240 to 300
seq_240to300 = bitome.sequence[3:60]
MGW_5mer = []
for i in range(3,60):
    kmer = bitome.sequence[i-2:i+3]
    sub_mgwtable = shape5mer_table['MGW'][shape5mer_table['MGW'][0] == str(kmer)]
    MGW_5mer.append( list(sub_mgwtable[1])[0] )
    
MGW_R = list(get_shape(3,60,shape_table)['MGW'])

In [12]:
bp = list(np.arange(3,60))
data = [bp,seq_240to300, MGW_5mer,MGW_R]
df = pd.DataFrame(data)
df.index = ['bp','seq','MGW_5mer','MGW_DNAshapeR']
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
bp,3,4,5,6,7,8,9,10,11,12,...,51,52,53,54,55,56,57,58,59,NaN
seq,T,T,T,T,C,A,T,T,C,T,...,A,A,G,A,G,T,G,T,C,NaN
MGW_5mer,4.03,3.68,4.35,5.4,5.5,4.46,4.39,4.43,5.14,5.47,...,3.68,4.02,4.68,4.86,4.78,5.1,5.49,4.99,5.22,NaN
MGW_DNAshapeR,4.14,4.03,3.68,4.35,5.4,5.5,4.46,4.39,4.43,5.14,...,3.38,3.68,4.02,4.68,4.86,4.78,5.1,5.49,4.99,5.22


In [31]:
get_shape(3,4,shape_table)

,HelT,Roll,Shift,Slide,Rise,TilT,Buckle,Shear,Stretch,Stagger,Opening,MGW,ProT,EP,loc
3,35.265,-3.975,-0.17,-1.805,3.510,0.96,-3.03,0.25,-0.02,0.12,0.37,4.14,-1.56,-6.03,3
4,34.675,-4.685,-0.26,-1.520,3.355,1.87,-8.27,-0.13,0.00,0.08,-0.74,4.03,-10.58,-8.76,4


In [19]:
bitome.sequence[0:6]

Seq('AGCTTT')

In [20]:
shape5mer_table['MGW'][shape5mer_table['MGW'][0] == 'AGCTT']

,0,1
159,AGCTT,4.14


In [34]:
# test for inter bp shape
HelT_5mer = []
for i in range(3,60):
    kmer = bitome.sequence[i-2:i+3]
    sub_helttable = shape5mer_table['HelT'][shape5mer_table['HelT'][0] == str(kmer)]
    HelT_5mer.append( 0.5*list(sub_helttable[1])[0] + 0.5*list(sub_helttable[3])[0] )
    
HelT_R = list(get_shape(3,60,shape_table)['HelT'])
data = [bp,seq_240to300, HelT_5mer,HelT_R ]
df1 = pd.DataFrame(data)
df1.index = ['bp','seq','HelT_5mer','HelT_DNAshapeR']
df1

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
bp,3,4,5,6,7,8,9,10,11,12,...,51,52,53,54,55,56,57,58,59,NaN
seq,T,T,T,T,C,A,T,T,C,T,...,A,A,G,A,G,T,G,T,C,NaN
HelT_5mer,34.685,37.1,36.66,35.395,35.21,33.925,34.16,36.355,34.025,32.83,...,37.1,34.935,34.115,33.71,32.955,34.64,34.475,35.425,33.93,NaN
HelT_DNAshapeR,35.265,34.675,36.94,36.56,35.625,35.07,33.89,34.325,36.2,33.985,...,37.7,37.23,34.845,34.27,33.78,33.08,34.58,34.485,35.335,33.91


In [36]:
bitome.sequence[0:10]

Seq('AGCTTTTCAT')

In [38]:
shape5mer_table['HelT'][shape5mer_table['HelT'][0] == 'AGCTT']

,0,1,2,3,4
159,AGCTT,38.05,0.0,32.14,0.0


In [39]:
shape5mer_table['HelT'][shape5mer_table['HelT'][0] == 'GCTTT']

,0,1,2,3,4
639,GCTTT,32.81,0.0,36.56,0.0


In [40]:
shape5mer_table['HelT'][shape5mer_table['HelT'][0] == 'CTTTT']

,0,1,2,3,4
511,CTTTT,37.18,0.0,37.02,0.0


## create gene operon tu table

In [24]:
data = []
for g in bitome.genes:
    if len(g.transcription_units) < 1:
        continue
    for tu in g.transcription_units:
        if tu.operon != None:
            row = {'gene_name': g.name, 'gene_id': g.locus_tag, 'tu_name':tu.name, 'tu_id':tu.id,
                  'operon_name':tu.operon.name, 'operon_id': tu.operon.id }
            data.append(row)
got_table = pd.DataFrame(data)

In [29]:
got_table.to_csv('data/Fur_multistrain_SN/gene_tu_operon_mapping.csv')